# Sistema de Informação de Agravos de Notificação (SINAN)

Esse notebook tem como objetivo realizar o ELT dos dados provenientes do SINAN que serão utilizados no projeto Dengue.

### Libs

In [1]:
%run ../config/bootstrap.py

In [2]:
import pandas as pd
from datetime import datetime 
from utils import get_project_root, save_parquet_in_chunks  
import basedosdados as bd

In [3]:
project_root = get_project_root() 
billing_id = 'trabalho-pratico-ml'

In [4]:
%load_ext sql
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaylimit = 100
%sql bigquery://trabalho-pratico-ml

E0000 00:00:1762630666.165237   85719 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Connecting to 'bigquery://trabalho-pratico-ml'

# ⌛ Staging
 

In [5]:
%%sql
SELECT * FROM `basedosdados.br_ms_sinan.dicionario` limit 10

Running query in 'bigquery://trabalho-pratico-ml'

10 rows affected.

,id_tabela,nome_coluna,chave,cobertura_temporal,valor
0,microdados_dengue,escolaridade_paciente,1,(1),1ª a 4ª série incompleta do EF
1,microdados_dengue,escolaridade_paciente,2,(1),4ª série completa do EF ( antigo 1° grau)
2,microdados_dengue,escolaridade_paciente,3,(1),5ª à 8ª série incompleta do EF (antigo ginásio ou 1° grau)
3,microdados_dengue,raca_cor_paciente,3,(1),Amarela
4,microdados_dengue,idade_paciente,4,(1),Ano
5,microdados_dengue,raca_cor_paciente,1,(1),Branca
6,microdados_dengue,criterio_confirmacao,2,(1),Clínico Epidemiológico
7,microdados_dengue,evolucao_caso,1,(1),Cura
8,microdados_dengue,idade_paciente,2,(1),Dia
9,microdados_dengue,escolaridade_paciente,8,(1),Educação superior completa


# Notifications

In [6]:
%%sql  
SELECT ano, count(*) as qt FROM `basedosdados.br_ms_sinan.microdados_dengue` group by 1 order by 1 

Running query in 'bigquery://trabalho-pratico-ml'

26 rows affected.

,ano,qt
0,2000,172855
1,2001,488590
2,2002,897093
3,2003,416609
4,2004,136867
5,2005,261501
6,2006,411022
7,2007,717097
8,2008,919324
9,2009,600658


# bi_query

### 2021

In [7]:
%%sql df <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2021

Running query in 'bigquery://trabalho-pratico-ml'

1010359 rows affected.

In [8]:
save_parquet_in_chunks(df, "data/4_gold/sinan/fat_sinan/2021")

Salvando arquivos Parquet:   0%|          | 0/1010359 [00:00<?, ?linhas/s]

✔️ 2021_part_1.parquet salvo com 5.42 MB (1010359 linhas)
✅ Todos os arquivos salvos com sucesso.


## 2022

In [9]:
%%sql df <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2022

Running query in 'bigquery://trabalho-pratico-ml'

1393877 rows affected.

In [10]:
save_parquet_in_chunks(df, "data/4_gold/sinan/fat_sinan/2022", 50, compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/1393877 [00:00<?, ?linhas/s]

✔️ 2022_part_1.parquet salvo com 5.32 MB (1393877 linhas)
✅ Todos os arquivos salvos com sucesso.


## 2023

In [11]:
%%sql df <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2023

Running query in 'bigquery://trabalho-pratico-ml'

1508653 rows affected.

In [ ]:
save_parquet_in_chunks(df, "data/4_gold/sinan/fat_sinan/2023", 50, compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/1508653 [00:00<?, ?linhas/s]

✔️ 2023_part_1.parquet salvo com 5.23 MB (1508653 linhas)
✅ Todos os arquivos salvos com sucesso.


: 

## 2024

In [ ]:
%%sql df <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2024 and data_notificacao<'2024-07-01'


Running query in 'bigquery://trabalho-pratico-ml'

6084673 rows affected.

In [ ]:
save_parquet_in_chunks(df,"data/4_gold/sinan/fat_sinan/2024_1",50,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/6078756 [00:00<?, ?linhas/s]

✔️ 2024_1_part_1.parquet salvo com 50.00 MB (2830107 linhas)
✔️ 2024_1_part_2.parquet salvo com 50.00 MB (2825650 linhas)
✔️ 2024_1_part_3.parquet salvo com 7.58 MB (422999 linhas)
✅ Todos os arquivos salvos com sucesso.


In [ ]:
del(df)

In [ ]:
%%sql df <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2024 and data_notificacao>='2024-07-01'

Running query in 'bigquery://ufrgs-ppgc-dengue'

349199 rows affected.

In [ ]:
save_parquet_in_chunks(df, "data/4_gold/sinan/fat_sinan/2024_2", 50, compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/349199 [00:00<?, ?linhas/s]

✔️ 2024_2_part_1.parquet salvo com 5.57 MB (349199 linhas)
✅ Todos os arquivos salvos com sucesso.


## 2025

In [ ]:
%%sql df <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2025

Running query in 'bigquery://ufrgs-ppgc-dengue'

1616395 rows affected.

In [ ]:
save_parquet_in_chunks(df, "data/4_gold/sinan/fat_sinan/2025", 50, compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/1616395 [00:00<?, ?linhas/s]

✔️ 2025_part_1.parquet salvo com 29.53 MB (1616395 linhas)
✅ Todos os arquivos salvos com sucesso.


In [ ]:
# pip install pandas pyarrow requests
# import requests, io, pandas as pd

# owner = "mlab-rs-brazil"
# repo = "dengue"
# path = "data/4_gold/sinan/fat_sinan"

# api = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
# resp = requests.get(api)
# resp.raise_for_status()

# dfs = []
# for item in resp.json():
#     # pega só arquivos .parquet (ignora diretórios)
#     if item.get("type") == "file" and item["name"].endswith(".parquet"):
#         url = item["download_url"]  # link raw
#         r = requests.get(url)
#         r.raise_for_status()
#         buf = io.BytesIO(r.content)
#         df_part = pd.read_parquet(buf, engine="pyarrow")
#         dfs.append(df_part)

# df = pd.concat(dfs, ignore_index=True)
# print(df.shape, df.head())


(11954197, 141)   tipo_notificacao id_agravo data_notificacao id_regional_saude_notificacao  \
0                2       A90       2021-02-05                      1353       
1                2       A90       2021-03-23                      1337       
2                2       A90       2021-01-09                      1352       
3                2       A90       2021-03-02                      1344       
4                2       A90       2021-03-13                      1336       

  id_municipio_notificacao  id_cnes data_primeiros_sintomas semana_sintomas  \
0                  3554003  6313760              2021-02-01         2021-05   
1                  3540705  5704790              2021-03-20         2021-11   
2                  3538006  7755333              2021-01-03         2021-01   
3                  3529005  2053799              2021-02-24         2021-08   
4                  3511003  2079216              2021-03-12         2021-10   

  pais_residencia  ano_nascimento_